## Image Classification

### Initialize pretrained model

In [ ]:
import keras
from keras.api.models import load_model


def initialize_model(model_path: str) -> keras.Model:
    model = load_model(model_path)
    return model


model = initialize_model("../tmp/identiface.h5")
model.summary()

### Preprocess facial image dataset

In [ ]:
import os

import numpy as np
import pandas as pd


def load_utkface_dataset(
    dataset_path: str,
    n: int = -1,
    seed: int = 42,
    gender_map: dict = {0: "MALE", 1: "FEMALE"},
) -> pd.DataFrame:
    try:
        paths = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith(".jpg")]

        np.random.seed(seed)
        np.random.shuffle(paths)

        if n > 0:
            paths = paths[:n]

        data = []
        for path in paths:
            try:
                filename = os.path.basename(path).split(".")[0]
                _, gender, *_ = filename.split("_")
                gender_val = int(gender)
                if gender_map:
                    gender_val = gender_map.get(gender_val, gender_val)
                data.append([path, gender_val])
            except (ValueError, IndexError):
                continue

        return pd.DataFrame(data, columns=["imagePath", "trueGender"])
    except Exception as e:
        raise RuntimeError(f"Failed to load dataset from {dataset_path}: {str(e)}")


df = load_utkface_dataset("../images/utkface", n=200)
df.head()

In [ ]:
from PIL import Image
from tqdm import tqdm


tqdm.pandas()


def load_image_array(
    path: str,
    color_mode: str = "L",
    target_size: tuple[int, int] = (48, 48),
    expand_dims: bool = True,
) -> np.ndarray:
    try:
        with Image.open(path) as image:
            image = image.convert(color_mode).resize(target_size)
            image_array = np.array(image, dtype=np.float32) / 255.0

            if color_mode == "L" and expand_dims:
                image_array = np.stack([image_array] * 3, axis=-1)

            return image_array
    except Exception as e:
        raise RuntimeError(f"Failed to load image {path}: {str(e)}")


df.loc[:, "imageArray"] = df["imagePath"].progress_apply(
    load_image_array,
    expand_dims=False,  # Identiface model expects single-channel images
)
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def display_images(
    df: pd.DataFrame,
    rows: int = 3,
    cols: int = 5,
    seed: int = 42,
    image_col: str = "imageArray",
    heatmap_col: str = None,
):
    n_images = rows * cols
    sample_df = df.sample(n=n_images, random_state=seed)

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    axes_flat = axes.flatten() if n_images > 1 else [axes]

    for idx, (_, row) in enumerate(sample_df.iterrows()):
        if idx >= n_images:
            break

        axes_flat[idx].imshow(row[image_col], cmap="gray")
        if heatmap_col and row[heatmap_col] is not None:
            axes_flat[idx].imshow(row[heatmap_col], cmap="jet", alpha=0.5)
        axes_flat[idx].axis("off")

    for idx in range(len(sample_df), len(axes_flat)):
        axes_flat[idx].axis("off")

    plt.tight_layout()
    plt.show()


display_images(df)

### Classify facial images

In [ ]:
def predict_gender(
    model: keras.Model,
    image: np.ndarray,
    gender_map: dict = {0: "MALE", 1: "FEMALE"},
) -> int:
    try:
        prediction = model.predict(np.expand_dims(image, axis=0), verbose=0)
        pred_class = np.argmax(prediction, axis=1)[0]
        return gender_map.get(pred_class, pred_class) if gender_map else pred_class
    except Exception as e:
        raise RuntimeError(f"Prediction failed: {str(e)}")


df["predictedGender"] = df["imageArray"].progress_apply(
    lambda x: predict_gender(
        model,
        x,
        {0: "FEMALE", 1: "MALE"},  # Identiface model predicts opposite
    )
)

In [ ]:
from sklearn.metrics import confusion_matrix


def plot_confusion_matrix(
    data: pd.DataFrame,
    true_col: str = "trueGender",
    predicted_col: str = "predictedGender",
    gender_map: dict = {0: "MALE", 1: "FEMALE"},
):
    labels = [gender_map[i] for i in sorted(gender_map.keys())]
    cm = confusion_matrix(data[true_col], data[predicted_col])

    plt.figure(figsize=(5, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels, cbar=False)

    plt.title("Confusion Matrix of Gender Classification")
    plt.xlabel("Predicted Gender")
    plt.ylabel("Actual Gender")
    plt.tight_layout()
    plt.show()


plot_confusion_matrix(df)

## Visual Explanation

### Generate class activation maps

In [ ]:
import tensorflow as tf
import tf_keras_vis
import tf_keras_vis.gradcam
import tf_keras_vis.scorecam


def generate_heatmap(
    model: keras.Model,
    image: np.ndarray,
    target_class: int,
    method: str = "gradcam++",
    gender_map: dict = {0: "MALE", 1: "FEMALE"},
    penultimate_layer: int = -1,
) -> np.ndarray:
    def model_modifier(cloned_model: tf.keras.Model) -> None:
        cloned_model.layers[penultimate_layer].activation = tf.keras.activations.linear

    def score_function(output: tf.Tensor) -> tf.Tensor:
        class_key = [k for k, v in gender_map.items() if v == target_class][0]
        return output[0][class_key]

    supported_methods = {
        "gradcam": tf_keras_vis.gradcam.Gradcam,
        "gradcam++": tf_keras_vis.gradcam.GradcamPlusPlus,
        "scorecam": tf_keras_vis.scorecam.Scorecam,
    }

    try:
        visualizer_class = supported_methods.get(method.lower())
        if not visualizer_class:
            raise ValueError(f"Unsupported method: {method}. Choose from {list(supported_methods.keys())}")
        visualizer = visualizer_class(model, model_modifier=model_modifier, clone=True)

        X = image if image.ndim == 3 else image[..., np.newaxis]  # Ensure image has channel dimension

        return visualizer(score_function, X, penultimate_layer=penultimate_layer)[0]

    except Exception as e:
        print(f"Failed to generate heatmap: {str(e)}")
        return np.zeros_like(image)


df["activation"] = df.progress_apply(
    lambda row: generate_heatmap(model=model, image=row["imageArray"], target_class=row["trueGender"]),
    axis=1,
)
display_images(df, heatmap_col="activation")

### Binarize saliency maps

In [ ]:
import skimage


def binarize_heatmap(
    heatmap: np.ndarray,
    top_percentile: int = 90,
    method: str = "otsu",
) -> np.ndarray:
    threshold_value = np.percentile(heatmap, top_percentile)
    heatmap[heatmap < threshold_value] = 0

    supported_methods = {
        "otsu": skimage.filters.threshold_otsu,
        "li": skimage.filters.threshold_li,
        "yen": skimage.filters.threshold_yen,
    }

    threshold_func = supported_methods.get(method)
    if not threshold_func:
        raise ValueError(f"Unsupported method: {method}. Choose from {list(supported_methods.keys())}")

    threshold_value = threshold_func(heatmap)
    return heatmap > threshold_value


df.loc[:, "activationBinary"] = df["activation"].progress_apply(binarize_heatmap)
display_images(df, 10, 10, heatmap_col="activationBinary")

### Generate activation boxes

In [ ]:
from skimage.measure import label, regionprops


def get_boxes(heatmap: np.ndarray) -> list[dict[str, int]]:
    labeled_heatmap = label(heatmap)
    regions = regionprops(labeled_heatmap)

    boxes = []
    for region in regions:
        min_row, min_col, max_row, max_col = region.bbox
        boxes.append({"min_x": min_col, "min_y": min_row, "max_x": max_col, "max_y": max_row, "label": None})

    return boxes


df.loc[:, "activationBox"] = df["activationBinary"].progress_apply(get_boxes)

In [ ]:
import cv2


def display_images_with_boxes(
    df: pd.DataFrame,
    rows: int = 3,
    cols: int = 5,
    seed: int = 42,
    image_col: str = "imageArray",
    box_col: str = "activationBox",
) -> None:
    n_images = rows * cols
    sample_df = df.sample(n=n_images, random_state=seed)

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    axes_flat = axes.flatten() if n_images > 1 else [axes]

    region_colors = {
        "left_eye": (0, 0, 1),
        "right_eye": (0, 1, 0),
        "nose": (1, 0, 0),
        "lips": (1, 1, 0),
        "left_cheek": (1, 0, 1),
        "right_cheek": (0, 1, 1),
        "left_eyebrow": (0.5, 0.5, 0.5),
        "right_eyebrow": (0.5, 0, 0.5),
    }

    for idx, (_, row) in enumerate(sample_df.iterrows()):
        if idx >= n_images:
            break

        image = row[image_col].copy()
        image = image if len(image.shape) == 3 else cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)  # Ensure RGB image

        for box in row[box_col]:
            min_x, min_y, max_x, max_y, label = box.values()
            cv2.rectangle(image, (min_x, min_y), (max_x, max_y), region_colors[label] if label else (1, 0, 0), 1)

        axes_flat[idx].imshow(image, cmap="gray")
        axes_flat[idx].axis("off")

    for idx in range(len(sample_df), len(axes_flat)):
        axes_flat[idx].axis("off")

    plt.tight_layout()
    plt.show()


display_images_with_boxes(df)

### Find all landmarks

In [ ]:
import mediapipe
from mediapipe.tasks.python.core.base_options import BaseOptions
from mediapipe.tasks.python.vision.face_landmarker import FaceLandmarker, FaceLandmarkerOptions


base_options = BaseOptions(model_asset_path="../tmp/mediapipe_landmarker.task")
options = FaceLandmarkerOptions(base_options=base_options, num_faces=1)
detector = FaceLandmarker.create_from_options(options)


def detect_landmark_points(
    image_path: str,
    detector=None,
    scale: int = 1,
) -> list[tuple[int, int]]:

    image = mediapipe.Image.create_from_file(str(image_path))
    detection_result = detector.detect(image)
    faces = detection_result.face_landmarks

    if len(faces) == 0:
        print(f"No face detected for {image_path}")
        return []

    return [(int(round(landmark.x * scale)), int(round(landmark.y * scale))) for landmark in faces[0]]


df.loc[:, "landmarkPoint"] = df["imagePath"].progress_apply(
    detect_landmark_points,
    scale=48,
    detector=detector,
)

In [ ]:
# fmt:off
landmark_map = {
    "left_eye": [249, 263, 362, 373, 374, 380, 381, 382, 384, 385, 386, 387, 388, 390, 398, 466],
    "right_eye": [7, 33, 133, 144, 145, 153, 154, 155, 157, 158, 159, 160, 161, 163, 173, 246],
    "nose": [1, 2, 4, 5, 6, 19, 45, 48, 64, 94, 97, 98, 115, 168, 195, 197, 220, 275, 278, 294, 326, 327, 344, 440],
    "lips": [0, 13, 14, 17, 37, 39, 40, 61, 78, 80, 81, 82, 84, 87, 88, 91, 95, 146, 178, 181, 185, 191, 267, 269, 270, 291, 308, 310, 311, 312, 314, 317, 318, 321, 324, 375, 402, 405, 409, 415],
    "left_cheek": [454, 447, 345, 346, 347, 330, 425, 427, 434, 416, 435, 288, 361, 323, 280, 352, 366, 411, 376, 401, 433],
    "right_cheek": [234, 227, 116,117,118, 101, 205, 207, 214, 192, 215, 58, 132, 93, 127, 50, 123, 137, 177, 147, 187, 213],
    "left_eyebrow": [276, 282, 283, 285, 293, 295, 296, 300, 334, 336],
    "right_eyebrow": [46, 52, 53, 55, 63, 65, 66, 70, 105, 107],
}
# fmt:on

def get_landmark_boxes(
    points: list[tuple[int, int]],
    landmark_map: dict[str, list[int]],
) -> list[dict[str, int]]:

    if not points:
        return []

    boxes = []

    for landmark, indices in landmark_map.items():
        x_coords, y_coords = zip(*[points[idx] for idx in indices])
        min_x, min_y = min(x_coords), min(y_coords)
        max_x, max_y = max(x_coords), max(y_coords)
        boxes.append({"min_x": min_x, "min_y": min_y, "max_x": max_x, "max_y": max_y, "label": landmark})

    return boxes


df.loc[:, "landmarkBox"] = df["landmarkPoint"].progress_apply(get_landmark_boxes, landmark_map=landmark_map)
display_images_with_boxes(df, box_col="landmarkBox")

### Find nearest landmarks to activation boxes

In [ ]:
def annotate_activation_boxes(
    activation_boxes: list[dict[str, int]],
    landmark_boxes: list[dict[str, int]],
) -> list[dict[str, int]]:
    if not landmark_boxes or not activation_boxes:
        return []

    boxes = []

    for a_box in activation_boxes:
        nearest_l_box = min(landmark_boxes, key=lambda l_box: compute_euclidean_distance(a_box, l_box))

        if has_significant_overlap(a_box, nearest_l_box, 0.2):
            a_box["label"] = nearest_l_box["label"]
            boxes.append(nearest_l_box)

    return boxes


# TODO: Use `scipy.spatial.distance.cdist` to compute pairwise distances between activation and landmark boxes
# TODO: Set distance metric as a parameter to the function
def compute_euclidean_distance(
    activation_box: dict[str, int],
    landmark_box: dict[str, int],
) -> float:
    x1 = (activation_box["min_x"] + activation_box["max_x"]) // 2
    y1 = (activation_box["min_y"] + activation_box["max_y"]) // 2
    x2 = (landmark_box["min_x"] + landmark_box["max_x"]) // 2
    y2 = (landmark_box["min_y"] + landmark_box["max_y"]) // 2
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def has_significant_overlap(
    activation_box: dict[str, int],
    landmark_box: dict[str, int],
    percentage: float = 0.5,
) -> bool:
    # Calculate the intersection area
    x_left = max(activation_box["min_x"], landmark_box["min_x"])
    y_top = max(activation_box["min_y"], landmark_box["min_y"])
    x_right = min(activation_box["max_x"], landmark_box["max_x"])
    y_bottom = min(activation_box["max_y"], landmark_box["max_y"])

    if x_right < x_left or y_bottom < y_top:
        return False

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # Calculate the areas of the activation and landmark boxes
    activation_area = (activation_box["max_x"] - activation_box["min_x"]) * (
        activation_box["max_y"] - activation_box["min_y"]
    )
    landmark_area = (landmark_box["max_x"] - landmark_box["min_x"]) * (landmark_box["max_y"] - landmark_box["min_y"])

    # Calculate the overlap percentage
    overlap_percentage = intersection_area / float(activation_area)

    return overlap_percentage >= percentage


df.loc[:, "annotatedBox"] = df.progress_apply(
    lambda row: annotate_activation_boxes(row["activationBox"], row["landmarkBox"]),
    axis=1,
)
display_images_with_boxes(df, box_col="annotatedBox")

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer


def encode_facial_regions(df: pd.DataFrame) -> pd.DataFrame:
    mlb = MultiLabelBinarizer()
    mlb.fit([list(landmark_map.keys())])
    df.loc[:, "label"] = df["annotatedBox"].apply(lambda boxes: [box["label"] for box in boxes])
    label_columns = df["label"].apply(lambda x: pd.Series([1 if label in x else 0 for label in mlb.classes_]))
    label_columns.columns = mlb.classes_
    return pd.concat([df, label_columns], axis=1)


df = encode_facial_regions(df)
df.head()

## Bias Analysis

### Compute equalized odds score

In [ ]:
def compute_equalized_odds_score(
    data: pd.DataFrame,
    protected_col: str = "trueGender",
    true_col: str = "trueGender",
    pred_col: str = "predictedGender",
) -> float:
    gender_values = data[protected_col].unique().tolist()

    tpr: dict[str, float] = {}
    fpr: dict[str, float] = {}

    for gender in gender_values:
        group = data[data[protected_col] == gender]

        # Calculate true positive rate
        pos = group[group[true_col] == gender]
        if len(pos) > 0:
            tpr[gender] = len(pos[pos[pred_col] == gender]) / len(pos)
        else:
            tpr[gender] = 0.0

        # Calculate false positive rate
        neg = group[group[true_col] != gender]
        if len(neg) > 0:
            fpr[gender] = len(neg[neg[pred_col] == gender]) / len(neg)
        else:
            fpr[gender] = 0.0

    # Compare rates between gender groups
    tpr_diff = abs(tpr[gender_values[0]] - tpr[gender_values[1]])
    fpr_diff = abs(fpr[gender_values[0]] - fpr[gender_values[1]])

    return max(tpr_diff, fpr_diff)


equalized_odds_score = compute_equalized_odds_score(df)
equalized_odds_score

### Compute feature probabilities

In [ ]:
def compute_feature_probabilities(
    data: pd.DataFrame,
    feature: str,
    protected_col: str = "trueGender",
    true_col: str = "trueGender",
    pred_col: str = "predictedGender",
) -> dict[int, float]:
    gender_values = data[protected_col].unique().tolist()
    probabilities: dict[str, float] = {}

    for gender in gender_values:
        # Get misclassified cases for this gender
        misclassified = data[(data[protected_col] == gender) & (data[pred_col] != data[true_col])]

        total_cases = len(misclassified)
        if total_cases > 0:
            feature_present = misclassified[misclassified[feature] == 1].shape[0]
            probabilities[gender] = round(feature_present / total_cases, 3)
        else:
            probabilities[gender] = 0.0

    return probabilities


feature_probabilities = {feature: compute_feature_probabilities(df, feature) for feature in landmark_map.keys()}
feature_probabilities

### Compute feature specific bias scores

In [ ]:
def compute_feature_bias_scores(
    data: pd.DataFrame,
    feature: str,
    protected_col: str = "trueGender",
    true_col: str = "trueGender",
    pred_col: str = "predictedGender",
) -> float:
    gender_values = data[protected_col].unique().tolist()
    probs = compute_feature_probabilities(data, feature, protected_col, true_col, pred_col)
    return round(abs(probs[gender_values[0]] - probs[gender_values[1]]), 3)


feature_bias_scores = {feature: compute_feature_bias_scores(df, feature) for feature in landmark_map}
feature_bias_scores

### Compute overall bias score

In [ ]:
def compute_overall_bias_score(
    data: pd.DataFrame,
    features: list[str],
    protected_col: str = "trueGender",
    true_col: str = "trueGender",
    pred_col: str = "predictedGender",
) -> float:
    gender_values = data[protected_col].unique().tolist()
    bias_scores = [
        compute_feature_bias_scores(data, feature, protected_col, true_col, pred_col)
        for feature in features
    ]
    return round(np.mean(bias_scores) if bias_scores else 0.0, 3)


overall_bias_score = compute_overall_bias_score(df, list(landmark_map.keys()))
overall_bias_score